# Project 2 - Medical Imaging 

## Imports

In [1]:
import os 
import re 

import numpy as np 
import pandas as pd 
from PIL import Image 
from matplotlib import pyplot as plt 
import seaborn as sns 

import skimage




## Global Variables

In [2]:
# Initial parameters for running the notebook 
PREPROCESS = True 
COMPUTE_FEATURES = True 

In [6]:
# Paths to directories used 

ROOT_DIR = "../"
DATA_DIR = f"{ROOT_DIR}data/"
IM_DIR = f"{DATA_DIR}example_image/"
MASK_DIR = f"{DATA_DIR}example_segmentation/"
DIAGNOSIS_PATH = f"{DATA_DIR}example_ground_truth.csv"

In [7]:
print(IM_DIR)

../data/example_image/


In [13]:
# Load all filenames into a dictionary
FILENAMES = {}
FILENAMES['images'] = sorted([i for i in os.walk(IM_DIR)][0][2])
FILENAMES['masks'] = sorted([i for i in os.walk(MASK_DIR)][0][2])
FILENAMES['image_num'] = len(FILENAMES['images'])
FILENAMES['cropped_images'] = []

In [14]:
print(FILENAMES)

{'images': ['ISIC_0001769.jpg', 'ISIC_0001852.jpg', 'ISIC_0001871.jpg', 'ISIC_0003462.jpg', 'ISIC_0003539.jpg', 'ISIC_0003582.jpg', 'ISIC_0003657.jpg', 'ISIC_0003805.jpg', 'ISIC_0004337.jpg', 'ISIC_0006651.jpg', 'ISIC_0006671.jpg', 'ISIC_0006815.jpg', 'ISIC_0006914.jpg', 'ISIC_0007141.jpg', 'ISIC_0007156.jpg', 'ISIC_0007235.jpg', 'ISIC_0007241.jpg', 'ISIC_0007332.jpg', 'ISIC_0007344.jpg', 'ISIC_0007528.jpg', 'ISIC_0007796.jpg', 'ISIC_0008025.jpg', 'ISIC_0008524.jpg', 'ISIC_0009995.jpg', 'ISIC_0010459.jpg', 'ISIC_0012099.jpg', 'ISIC_0012109.jpg', 'ISIC_0012126.jpg', 'ISIC_0012127.jpg', 'ISIC_0012143.jpg', 'ISIC_0012151.jpg', 'ISIC_0012159.jpg', 'ISIC_0012160.jpg', 'ISIC_0012191.jpg', 'ISIC_0012201.jpg', 'ISIC_0012204.jpg', 'ISIC_0012206.jpg', 'ISIC_0012210.jpg', 'ISIC_0012221.jpg', 'ISIC_0012222.jpg', 'ISIC_0012254.jpg', 'ISIC_0012256.jpg', 'ISIC_0012288.jpg', 'ISIC_0012306.jpg', 'ISIC_0012313.jpg', 'ISIC_0012316.jpg', 'ISIC_0012335.jpg', 'ISIC_0012380.jpg', 'ISIC_0012383.jpg', 'ISIC_00

## Preprocessing

## Processing 

## Feature Extraction 

In [ ]:
## Find the area and perimeter of the mask 
def get_area_perimeter(mask): 
    area = 0 
    perimeter = 0 
    return area, perimeter 

In [ ]:
def get_compactness(mask): 
    area, perimenter = get_area_perimeter(mask) 

    return 

In [ ]:
def get_average_luminance(image): 

    return 

In [ ]:
def get_luminance_variability(image,measure="variance"): 

    return 

In [ ]:
def get_avg_color(image): 

    return 

In [ ]:
def get_color_variability(image,measure="variance"): 

    return 

In [ ]:
def get_assymetry(mask): 
    
    return 